<a href="https://colab.research.google.com/github/Aleksandr62aa/ML_DL/blob/main/FC_NN_class_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classification of wine bottles by a fully connected neural network**

**Importing libraries**

In [ ]:
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine

# Fixed seed

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True

**Train dataset**

In [ ]:
wine = load_wine()
features = 13 # use 13 features
wine.data.shape

(178, 13)

**Train Test dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :features],
    wine.target,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

**Model construction**


In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        # self.activ1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        # self.activ2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

n_input = features
n_hidden =  50
wine_net = WineNet(n_input, n_hidden)

**Optimizer**

In [ ]:
optimizer = torch.optim.Adam(wine_net.parameters(), lr=1.0e-3)
#optimizer = torch.optim.SGD(wine_net.parameters(), lr=1.0e-3)


**Loss function**

In [ ]:
loss = torch.nn.CrossEntropyLoss()
#loss = torch.nn.NLLLoss()

**Training procedure**

In [ ]:
batch_size =  8

for epoch in range(1000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]

        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]

        preds = wine_net.forward(x_batch)

        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()

    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.3519)
tensor(0.8889)
tensor(0.9074)
tensor(0.9259)
tensor(0.9074)
tensor(0.9444)
tensor(0.9259)
tensor(0.9444)
tensor(0.9630)
tensor(0.9815)


In [ ]:
print("features =", wine_net.fc1.in_features)
print("accuracy =", (test_preds == y_test).float().mean().numpy())

features = 13
accuracy = 0.9814815
